In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Descargar los datos diarios de BTC/USD
btc = yf.download('BTC-USD', interval='1d', start='2015-01-01')
btc = btc[['Close']].dropna()

# Función para calcular estrategia EMA
def backtest_ema(df, short_window, long_window):
    df = df.copy()
    df['EMA_short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df['EMA_long'] = df['Close'].ewm(span=long_window, adjust=False).mean()
    df['Signal'] = np.where(df['EMA_short'] > df['EMA_long'], 1, 0)
    df['Position'] = df['Signal'].shift(1)
    df['Daily Return'] = df['Close'].pct_change()
    df['Strategy Return'] = df['Daily Return'] * df['Position']
    cumulative_return = (df['Strategy Return'] + 1).prod()
    return cumulative_return, df

# Buscar mejor combinación de EMAs
best_result = 0
best_combo = (0, 0)
for short in range(5, 50, 5):
    for long in range(short + 5, 100, 5):
        ret, _ = backtest_ema(btc, short, long)
        if ret > best_result:
            best_result = ret
            best_combo = (short, long)

print(f"Mejor cruce de EMAs: EMA{best_combo[0]} y EMA{best_combo[1]} con retorno acumulado: {best_result:.2f}x")

# Aplicar mejor combinación y mostrar última señal
_, ema_df = backtest_ema(btc, *best_combo)
ultima_senal_ema = "COMPRAR" if ema_df['Signal'].iloc[-1] == 1 else "VENDER"
print(f"Última señal por cruce EMA: {ultima_senal_ema}")



[*********************100%***********************]  1 of 1 completed


Mejor cruce de EMAs: EMA20 y EMA70 con retorno acumulado: 590.85x
Última señal por cruce EMA: VENDER


In [4]:
# Aplicar mejor combinación y evaluar señal actual
_, ema_df = backtest_ema(btc, *best_combo)
ultima_senal_ema = "COMPRAR" if ema_df['Signal'].iloc[-1] == 1 else "VENDER"
print(f"📈 Última señal por cruce EMA: {ultima_senal_ema}")

# Cálculo de MACD
btc['EMA12'] = btc['Close'].ewm(span=12, adjust=False).mean()
btc['EMA26'] = btc['Close'].ewm(span=26, adjust=False).mean()
btc['MACD'] = btc['EMA12'] - btc['EMA26']
btc['Signal_MACD'] = btc['MACD'].ewm(span=9, adjust=False).mean()

# Señales de compra/venta MACD
btc['MACD_Signal'] = np.where(btc['MACD'] > btc['Signal_MACD'], 1, 0)
btc['MACD_Position'] = btc['MACD_Signal'].shift(1)
btc['Daily Return'] = btc['Close'].pct_change()
btc['MACD_Return'] = btc['Daily Return'] * btc['MACD_Position'].astype(float)
macd_ret = (btc['MACD_Return'] + 1).prod()

print(f"🔁 Retorno acumulado con estrategia MACD: {macd_ret:.2f}x")

ultima_senal_macd = "COMPRAR" if btc['MACD_Signal'].iloc[-1] == 1 else "VENDER"
print(f"📊 Última señal por MACD: {ultima_senal_macd}")

# Recomendación combinada
print("\n--- ✅ Recomendación Final ---")
if ultima_senal_ema == "COMPRAR" and ultima_senal_macd == "COMPRAR":
    print("✅ Señal combinada: COMPRAR")
elif ultima_senal_ema == "VENDER" and ultima_senal_macd == "VENDER":
    print("🚫 Señal combinada: VENDER")
else:
    print("🤔 Señal combinada: NEUTRAL / ESPERAR")

📈 Última señal por cruce EMA: VENDER
🔁 Retorno acumulado con estrategia MACD: 1.00x
📊 Última señal por MACD: VENDER

--- ✅ Recomendación Final ---
🚫 Señal combinada: VENDER
